<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from pprint import pprint
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017")
db = client.osm_data
sbg = db.SouthBG

In [2]:
##Display statistics about the database
# mongoschema is available via pip, see also https://github.com/nimeshkverma/mongo_schema/
# some modification were required to make mongoschema compatible with Python 3.6
from mongoschema.mongoschema import Schema
schema = Schema("osm_data", "SouthBG")
num_docs, result = schema.get_schema()
schema.print_schema()

Total number of docs : 1218483
+-------------------------------+------------------+-----------------------+----------------+-----------------------+
|              Key              | Occurrence Count | Occurrence Percentage |   Value Type   | Value Type Percentage |
+-------------------------------+------------------+-----------------------+----------------+-----------------------+
|              _id              |     1218483      |         100.0         |     other      |         100.0         |
|             class             |     1218483      |         100.0         | <class 'str'>  |         100.0         |
|            osm_base           |        1         |          0.0          | <class 'str'>  |         100.0         |
|              pos              |     1097260      |         90.05         | <class 'list'> |         100.0         |
|               id              |     1218479      |         100.0         | <class 'str'>  |         100.0         |
|            created     

In [3]:
# Set convenience constants
EXISTS = {"$exists": True}
NOT_NULL = lambda x: {"$ifNull": [ x, 0 ]}
NOT_NULL("test")

{'$ifNull': ['test', 0]}

In [4]:
# Count unique useres
len(sbg.distinct("created.uid"))

815

In [5]:
# Count different classes of documents
pipeline = [{"$group": { "_id": "$class",
                        "count": {"$sum": 1}}},
            {"$sort": {"count": -1}}]
list(sbg.aggregate(pipeline))

[{'_id': 'node', 'count': 1097260},
 {'_id': 'way', 'count': 120256},
 {'_id': 'relation', 'count': 963},
 {'_id': 'bounds', 'count': 1},
 {'_id': 'osm', 'count': 1},
 {'_id': 'meta', 'count': 1},
 {'_id': 'note', 'count': 1}]

In [6]:
sbg.count_documents({"power": EXISTS})

11838

In [7]:
## Question 3.1 - Explore types fields related to power
pipeline = [{"$match": {"power": EXISTS}},
            {"$group": { "_id": "$power",
                        "count": {"$sum": 1}}},
            {"$sort": {"count": -1}}]
list(sbg.aggregate(pipeline))

[{'_id': 'tower', 'count': 5873},
 {'_id': 'pole', 'count': 4714},
 {'_id': 'minor_line', 'count': 662},
 {'_id': 'transformer', 'count': 292},
 {'_id': 'line', 'count': 185},
 {'_id': 'substation', 'count': 74},
 {'_id': 'generator', 'count': 27},
 {'_id': 'plant', 'count': 5},
 {'_id': 'portal', 'count': 3},
 {'_id': 'terminal', 'count': 1},
 {'_id': 'yes', 'count': 1},
 {'_id': 'pole;tower', 'count': 1}]

In [8]:
## Question 3.1 - Count source of the power when plant or generator
pipeline = [{"$match": {"$or":[ {"power": 'generator'},
                                {"power": 'plant'}]}},
            {"$project":  {"power": 1,
                           "source": {"$ifNull": ["$generator.source",
                                                "$plant.source"]}}},
            {"$group": { "_id": "$source",
                        "count": {"$sum": 1}}},
            {"$sort": {"count": -1}}]
list(sbg.aggregate(pipeline))

[{'_id': 'solar', 'count': 17},
 {'_id': 'hydro', 'count': 5},
 {'_id': 'coal', 'count': 5},
 {'_id': None, 'count': 4},
 {'_id': 'gas', 'count': 1}]

In [9]:
## Question 3.1 - Explore if it is possible to calculate the total output of each source
pipeline = [{"$match": {"$or":[ {"power": 'generator'},
                                {"power": 'plant'}]}},
            {"$project":  {"power": 1,
                           "name": 1,
                           "source": {"$ifNull": ["$generator.source",
                                                "$plant.source"]},
                          "output": {"$ifNull": ["$generator.output.electricity",
                                                "$plant.output.electricity"]}}}]
list(sbg.aggregate(pipeline))

[{'_id': ObjectId('5f2d4c56f80d478f6f221dcf'),
  'name': {'local': 'ВЕЦ "Асеница"'},
  'power': 'generator',
  'source': 'hydro',
  'output': '7 MW'},
 {'_id': ObjectId('5f2d4c57f80d478f6f249958'),
  'name': {'local': 'ВЕЦ "Асеница 1"'},
  'power': 'generator',
  'source': 'hydro',
  'output': '7 MW'},
 {'_id': ObjectId('5f2d4c57f80d478f6f249959'),
  'name': {'local': 'ВЕЦ "Студен кладенец"'},
  'power': 'generator',
  'source': 'hydro',
  'output': '90 MW'},
 {'_id': ObjectId('5f2d4c59f80d478f6f29c7a6'),
  'name': {'local': 'Блок 1'},
  'power': 'generator',
  'source': 'coal',
  'output': '345 MW'},
 {'_id': ObjectId('5f2d4c59f80d478f6f29c7a7'),
  'name': {'local': 'Блок 2'},
  'power': 'generator',
  'source': 'coal',
  'output': '345 MW'},
 {'_id': ObjectId('5f2d4c5af80d478f6f2bdd72'),
  'name': {'local': 'МВЕЦ "Десислава"'},
  'power': 'generator',
  'source': 'hydro'},
 {'_id': ObjectId('5f2d4c5df80d478f6f31dfc5'),
  'name': {'local': 'ТЕЦ "Север"'},
  'power': 'generator',
  'so

In [10]:
##Question 3.2 Count the different entities which have a website
# different types of objects (with different schemas) can have a website
pipeline = [{"$match": {"class": "node",
                        "website": EXISTS}},
            {"$project": {"entity": {"$switch": {"branches": [  {"case": NOT_NULL("$amenity"),
                                                                 "then": "$amenity"},
                                                                {"case": NOT_NULL("$shop"),
                                                                 "then": {"$concat": [{"$cond": [{"$eq": ["$shop",
                                                                                                          'yes']},
                                                                                                 "",
                                                                                                 "$shop"]},
                                                                                      " ",
                                                                                      "shop"]}},
                                                                {"case": NOT_NULL("$place"),
                                                                 "then": "$place"},
                                                                {"case": NOT_NULL("$tourism"),
                                                                 "then": "$tourism"},
                                                                {"case": NOT_NULL("$office"),
                                                                 "then": "office"}],
                                                 "default": None}}}},
            {"$group": { "_id": "$entity",
                        "count": {"$sum": 1}}},
            {"$match": {"count": {"$gte": 5}}},
            {"$sort": {"count": -1}}]
list(sbg.aggregate(pipeline))

[{'_id': None, 'count': 73},
 {'_id': 'restaurant', 'count': 24},
 {'_id': 'clothes shop', 'count': 21},
 {'_id': 'office', 'count': 19},
 {'_id': 'dentist', 'count': 17},
 {'_id': 'hotel', 'count': 16},
 {'_id': 'car_repair shop', 'count': 11},
 {'_id': 'bank', 'count': 11},
 {'_id': ' shop', 'count': 9},
 {'_id': 'post_office', 'count': 5},
 {'_id': 'supermarket shop', 'count': 5}]

In [11]:
#Q3.2 further exploration - how many of the restaurants have website
pipeline = [{"$match": {"amenity": "restaurant"}},
            {"$project": { "has_website": {"$convert": {"input": "$website",
                                                        "to": "bool" }}}},
            {"$group": {"_id": "$has_website",
                        "count": {"$sum": 1}}},
            {"$project": {"has_website": "$_id", "count": 1, "_id": 0}}]
list(sbg.aggregate(pipeline))

[{'count': 27, 'has_website': True}, {'count': 352, 'has_website': None}]

In [12]:
#Q3.2 further exploration - what other entities have websites
list(sbg.find({"website": EXISTS,
               "amenity": {"$exists": False},
               "place": {"$exists": False},
               "amenity": {"$exists": False},
                "shop": {"$exists": False},
              "tourism": {"$exists": False},
              "office": {"$exists": False} }))

[{'_id': ObjectId('5f2d4c57f80d478f6f2510e6'),
  'class': 'node',
  'pos': [42.1510102, 24.7460763],
  'id': '2072147024',
  'created': {'version': '4',
   'timestamp': '2019-03-23T23:11:52Z',
   'changeset': '68457555',
   'uid': '9451067',
   'user': 'b-jazz-bot'},
  'address': {'housenumber': '53', 'postcode': 4000},
  'name': {'local': 'Idea Studio Ltd.'},
  'name_1': 'idea studio',
  'operator': 'Borislav Arapchev',
  'source': 'office',
  'website': 'https://www.ideabg.com/'},
 {'_id': ObjectId('5f2d4c5af80d478f6f2b3f56'),
  'class': 'node',
  'pos': [42.125432, 24.7411632],
  'id': '4432811902',
  'created': {'version': '1',
   'timestamp': '2016-10-05T14:40:00Z',
   'changeset': '42659407',
   'uid': '4678925',
   'user': 'BorjaBobby'},
  'address': {'city': 'Пловдив',
   'housenumber': '48',
   'street': 'бул. Македония'},
  'name': {'local': 'Автостъкла Кобра'},
  'phone': '+359884351001',
  'website': 'http://www.autoglass.bg/bg'},
 {'_id': ObjectId('5f2d4c5af80d478f6f2b4aeb

In [13]:
## Improvement 4.1
sbg.count_documents({"place": {"$exists": True}, "is_in":{"$exists": False}})

318

In [14]:
dict(sbg.find_one({"address": {"$exists": True}, "place": {"$exists": False}, "address.street": {"$exists": False}}))

{'_id': ObjectId('5f2d4c56f80d478f6f22da25'),
 'class': 'node',
 'pos': [41.932002, 25.5591893],
 'id': '1197123593',
 'created': {'version': '4',
  'timestamp': '2019-11-27T20:47:11Z',
  'changeset': '77652773',
  'uid': '9202927',
  'user': 'Araucaria_araucana'},
 'address': {'housenumber': '7-А'}}

In [15]:
## Improvement 4.2
node_id = dict(sbg.find_one({
                        "address": {"$exists": True}, 
                        "place": {"$exists": False}, 
                        "address.street": {"$exists": False}}))["id"]
list(sbg.find({
                    "class": "way",
                    "node_refs": {
                        "$elemMatch": {
                            "$eq": node_id
                        }}}))

[{'_id': ObjectId('5f2d4c5df80d478f6f31f364'),
  'class': 'way',
  'node_refs': ['1197126729',
   '1197123193',
   '2463960328',
   '1197126242',
   '1197125395',
   '1197126836',
   '1197122184',
   '2463960335',
   '1197123477',
   '1197128935',
   '1197125276',
   '1197126667',
   '2463960327',
   '1197128980',
   '1197127481',
   '1197129227',
   '1197123530',
   '1197127047',
   '1197128582',
   '1197125495',
   '1197127541',
   '1197123593',
   '1197123148',
   '1197129407',
   '1197126729'],
  'id': '103690885',
  'created': {'version': '4',
   'timestamp': '2019-12-13T15:32:05Z',
   'changeset': '78381805',
   'uid': '9516754',
   'user': 'lqlqlqlio'},
  'address': {'city': 'Хасково',
   'housenumber': '7',
   'postcode': 6300,
   'street': 'ул. Сан Стефано'},
  'building': {'type': 'apartments', 'levels': '4'},
  'old_name': 'Мебелен магазин'}]